In [67]:
import sqlite3
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [68]:
def read_sqlite_data():
    """
    Read the data using sqlite3 library in python
    """
    sqlite_data = sqlite3.connect("travel.sqlite")
    return sqlite_data

In [69]:
def get_table_list(sqlite_data):
    """
    Queries the names of all the tables in the sql database

    Returns:
        list: list of all the tables names
    """
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    cur = sqlite_data.cursor()
    cur.execute(sql_query)
    table_list = cur.fetchall()
    cleaned_table_list = [table_list[i][0] for i in range(len(table_list))]
    return cleaned_table_list

In [70]:
def create_dataframes_dict(sqlite_data, cleaned_table_list):
    """
    transforms all the tables of the database into pandas dataframes

    Args:
        sqlite_data (_type_): _description_
        cleaned_table_list (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Transforms the result of the sql query into a data frame
    dfs_dict = {cleaned_table_list[i]: pd.read_sql_query("SELECT * FROM "+cleaned_table_list[i], 
                                    sqlite_data) for i in range(len(cleaned_table_list))}
    return dfs_dict

In [71]:
def merge_dfs(dfs_dict):
    """
    returns a complete dataframe merging all the dataframes/tables based on their primary 
    and secondary keys

    Args:
        dfs_dict (dict): dictionary containing all the dataframes

    Returns:
        pandas df: df containing the merged data frame
    """
    all_tickets_data = dfs_dict['ticket_flights'].merge(dfs_dict['flights'], on = "flight_id").merge(
        dfs_dict["aircrafts_data"], on = "aircraft_code").merge(
        dfs_dict["airports_data"], left_on = "departure_airport", right_on = "airport_code", 
        suffixes=("", "_departure")).merge(dfs_dict["airports_data"], left_on = "arrival_airport", right_on = "airport_code", 
        suffixes=("", "_arrival")).merge(dfs_dict["tickets"], on = "ticket_no").merge(
        dfs_dict["bookings"], on= "book_ref"
        )
    return all_tickets_data

In [72]:
def remove_keys(all_tickets_data):
    """
    Remove the relational primary and secondary keys for SQL querying
    """
    cleaned_tickets_data = all_tickets_data.drop(
        ["flight_id", "aircraft_code", "airport_code", "airport_code_arrival",
        "ticket_no", "book_ref", "flight_no", "passenger_id"], axis = 1)
    return cleaned_tickets_data

In [73]:
def remove_irrelevent_columns(cleaned_tickets_data):
    """
    Dropping obviously irrelevent columns to the price of the ticket
    """
    relevent_tickets_data = cleaned_tickets_data.drop(["status", "timezone", "coordinates",
                                                        "coordinates_arrival", "timezone_arrival",
                                                        "actual_arrival", "actual_departure",
                                                        "total_amount"], axis = 1)
    return relevent_tickets_data

In [74]:
def rename_columns(relevent_tickets_data):
    """
    renaming columns for clarification
    """
    relevent_tickets_data = relevent_tickets_data.rename({"airport_name": "airport_name_departure", 
                                                            "city": "city_departure"}, axis = 1)
    return relevent_tickets_data

In [75]:
sqlite_data = read_sqlite_data()
cleaned_table_list = get_table_list(sqlite_data)
dfs_dict = create_dataframes_dict(sqlite_data, cleaned_table_list)
all_tickets_data = merge_dfs(dfs_dict)
cleaned_tickets_data = remove_keys(all_tickets_data)
relevent_tickets_data = remove_irrelevent_columns(cleaned_tickets_data)
relevent_tickets_data = rename_columns(relevent_tickets_data)

In [76]:
relevent_tickets_data.columns

Index(['fare_conditions', 'amount', 'scheduled_departure', 'scheduled_arrival',
       'departure_airport', 'arrival_airport', 'model', 'range',
       'airport_name_departure', 'city_departure', 'airport_name_arrival',
       'city_arrival', 'book_date'],
      dtype='object')

In [77]:
def convert_column_to_dict(data, column_names):
    """
    Convert column from str to dict
    """
    for column_name in column_names:
        data[column_name] = data[column_name].map(lambda x: json.loads(x))
    return data

In [78]:
def english_formatting(data, column_names):
    """
    pick only the english names
    """
    for column_name in column_names:
        data[column_name] = data[column_name].apply(lambda x: x['en'])
    return data

In [79]:
def date_formatting(data, column_names):
    """
    Correctly format the date columns
    """
    for column_name in column_names:
        data[column_name] = data[column_name].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))
    return data

In [80]:
formatted_tickets_data = convert_column_to_dict(relevent_tickets_data, ["model", 
                        "airport_name_departure", "city_departure", "airport_name_arrival",
                        "city_arrival"])

In [ ]:
en_formatted_tickets_data = english_formatting(formatted_tickets_data, ["model", 
                        "airport_name_departure", "city_departure", "airport_name_arrival",
                        "city_arrival"])

In [44]:
date_formatted_tickets_data = date_formatting(en_formatted_tickets_data, ["scheduled_departure", 
                                            "scheduled_arrival", "book_date"])

In [66]:
formatted_tickets_data["scheduled_departure"][0]

'2017-07-16 18:15:00+03'

In [46]:
def create_new_features(data):
    data

,amount,total_amount
0,42100,110300
1,13400,110300
2,14000,110300
3,13400,110300
4,14000,110300
...,...,...
1045721,22800,155700
1045722,132900,155700
1045723,22800,67100
1045724,44300,67100
